In [24]:
import pandas as pd
import numpy as np

# 1) Load your data into a DataFrame

df = pd.read_csv("/Users/sanjju/Downloads/repo-date.csv")


In [25]:

# 2) Parse dates
for c in ["BUSINESS_DATE", "START_DATE", "END_DATE"]:
    df[c] = pd.to_datetime(df[c], format="%m/%d/%Y")

# 3) Compute repoLen in days/360
df["repoLen"] = (df["END_DATE"] - df["START_DATE"]).dt.days / 360

# 4) For each term, treat the whole interval as one bucket:
#    expo = repoLen, bValue = COMPOSITE_REPO_RATE
df["tm_b_prod"] = df["repoLen"] * df["COMPOSITE_REPO_RATE"]

# 5) Final model rate
df["modelRate"] = (np.exp(df["tm_b_prod"]) - 1) / df["repoLen"]

print(df[[
    "START_DATE","END_DATE","repoLen",
    "COMPOSITE_REPO_RATE","tm_b_prod","modelRate"
]])

   START_DATE   END_DATE   repoLen  COMPOSITE_REPO_RATE  tm_b_prod  modelRate
0  2025-05-06 2025-05-07  0.002778             4.347362   0.012076   4.373717
1  2025-05-07 2025-05-08  0.002778             4.358350   0.012107   4.384839
2  2025-05-07 2025-05-09  0.005556             4.400000   0.024444   4.454219
3  2025-05-08 2025-05-09  0.002778             4.400001   0.012222   4.427000
4  2025-05-07 2025-05-13  0.016667             4.396716   0.073279   4.561817
5  2025-05-06 2025-05-16  0.027778             4.380000   0.121667   4.657593
6  2025-05-07 2025-05-20  0.036111             4.400000   0.158889   4.768829
7  2025-05-06 2025-05-21  0.041667             4.410067   0.183753   4.841249
8  2025-05-07 2025-05-21  0.038889             4.400000   0.171111   4.798867
9  2025-05-07 2025-05-27  0.055556             4.391244   0.243958   4.973233
10 2025-05-07 2025-05-28  0.058333             4.358000   0.254217   4.962020
11 2025-05-06 2025-05-30  0.066667             4.400000   0.2933

In [15]:
# Parse the date columns into datetime objects
for col in ["BUSINESS_DATE", "START_DATE", "END_DATE"]:
    df[col] = pd.to_datetime(df[col], format="%m/%d/%Y")

In [16]:
# 2) Compute repo length in "360-second blocks"
#    repoLen_i = (END_DATE_i - START_DATE_i).total_seconds() / 360
df["repoLen"] = (df["END_DATE"] - df["START_DATE"]).dt.days / 360

In [17]:
df["repoLen"]

0     0.002778
1     0.002778
2     0.005556
3     0.002778
4     0.016667
5     0.027778
6     0.036111
7     0.041667
8     0.038889
9     0.055556
10    0.058333
11    0.066667
12    0.063889
13    0.072222
14    0.083333
15    0.102778
16    0.277778
17    0.255556
18    0.169444
Name: repoLen, dtype: float64

In [18]:
# 3) Define your interval DataFrame (bValueDf).
#    Here, for simplicity, we treat each term as a single interval:
#       b_stDt = START_DATE_i, b_endDt = END_DATE_i
#       Value     = COMPOSITE_REPO_RATE_i
#    So NB = 1 for each term.
b_values = df["COMPOSITE_REPO_RATE"].values

In [19]:
# 4) Compute exposures vector expo(i):
#    Since NB = 1, expo[i] = repoLen[i]
expo = df["repoLen"].values

In [20]:

# 5) Dot‐product: tm_b_prod_i = expo[i] * b_values[i]
df["tm_b_prod"] = expo * b_values


In [21]:
# 6) Model rate:
#    modelRt_i = (exp(tm_b_prod_i) - 1) / repoLen_i
df["modelRate"] = (np.exp(df["tm_b_prod"]) - 1) / df["repoLen"]

In [22]:
# Final: Inspect the DataFrame
print(df[[
    "START_DATE", "END_DATE", "repoLen", 
    "COMPOSITE_REPO_RATE", "tm_b_prod", "modelRate"
]])


   START_DATE   END_DATE   repoLen  COMPOSITE_REPO_RATE  tm_b_prod  modelRate
0  2025-05-06 2025-05-07  0.002778             4.347362   0.012076   4.373717
1  2025-05-07 2025-05-08  0.002778             4.358350   0.012107   4.384839
2  2025-05-07 2025-05-09  0.005556             4.400000   0.024444   4.454219
3  2025-05-08 2025-05-09  0.002778             4.400001   0.012222   4.427000
4  2025-05-07 2025-05-13  0.016667             4.396716   0.073279   4.561817
5  2025-05-06 2025-05-16  0.027778             4.380000   0.121667   4.657593
6  2025-05-07 2025-05-20  0.036111             4.400000   0.158889   4.768829
7  2025-05-06 2025-05-21  0.041667             4.410067   0.183753   4.841249
8  2025-05-07 2025-05-21  0.038889             4.400000   0.171111   4.798867
9  2025-05-07 2025-05-27  0.055556             4.391244   0.243958   4.973233
10 2025-05-07 2025-05-28  0.058333             4.358000   0.254217   4.962020
11 2025-05-06 2025-05-30  0.066667             4.400000   0.2933

In [23]:
df

,BUSINESS_DATE,START_DATE,END_DATE,COMPOSITE_REPO_RATE,repoLen,tm_b_prod,modelRate
0,2025-05-06,2025-05-06,2025-05-07,4.347362,0.002778,0.012076,4.373717
1,2025-05-06,2025-05-07,2025-05-08,4.358350,0.002778,0.012107,4.384839
2,2025-05-06,2025-05-07,2025-05-09,4.400000,0.005556,0.024444,4.454219
3,2025-05-06,2025-05-08,2025-05-09,4.400001,0.002778,0.012222,4.427000
4,2025-05-06,2025-05-07,2025-05-13,4.396716,0.016667,0.073279,4.561817
5,2025-05-06,2025-05-06,2025-05-16,4.380000,0.027778,0.121667,4.657593
6,2025-05-06,2025-05-07,2025-05-20,4.400000,0.036111,0.158889,4.768829
7,2025-05-06,2025-05-06,2025-05-21,4.410067,0.041667,0.183753,4.841249
8,2025-05-06,2025-05-07,2025-05-21,4.400000,0.038889,0.171111,4.798867
9,2025-05-06,2025-05-07,2025-05-27,4.391244,0.055556,0.243958,4.973233
